# Geolocation

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

# Autores

In [2]:
%%time

autores_df = pd.read_csv('autores6.csv', header=0)
autores_df = autores_df.fillna('')

CPU times: user 202 ms, sys: 4.49 ms, total: 206 ms
Wall time: 205 ms


In [3]:
autores_df.shape

(76722, 7)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR
0,Adivina en qué pais ...,Spain,Martina Badstuber,Germany,2010,Berlin,Madrid
1,¡No quiero hacer pipí en el orinal!,Spain,Roser Rius,Mexico,2009,Mexico City,Madrid
2,Humo,Spain,Antón Fortes Torres,Spain,2008,Madrid,Madrid
3,Galleta para perros,Spain,Helen Cooper,United Kingdom,2008,London,Madrid
4,Autobio,Spain,Cyril Pedrosa,France,2009,Paris,Madrid


# Geocoordinates

In [5]:
autores_df['CAPITAL_EJEMPLAR'].nunique()

117

In [6]:
autores_df['CAPITAL_AUTOR'].nunique()

204

In [7]:
places = set(autores_df['CAPITAL_AUTOR'].unique()).union(autores_df['CAPITAL_EJEMPLAR'].unique())

In [8]:
len(places)

249

In [9]:
google_maps = GoogleMaps(api_key=KEY)

In [10]:
%%time

locs = {} # (lat, lng)

for x in places:
    try:
        locations = google_maps.search(location=x)
        if len(locations.list_data) > 1:
            print('MORE THAN ONE LOCATION:', x)
        location = locations.first()
        locs[x] = (location.lat, location.lng)
    except:
        print('NOT LOCATED:', x)

NOT LOCATED: Jakarta
NOT LOCATED: Batavia, Dutch East Indies
NOT LOCATED: St. John's
NOT LOCATED: Singapore
NOT LOCATED: Kingdom of England
NOT LOCATED: Gibraltar
NOT LOCATED: Brussels
NOT LOCATED: Hong Kong
NOT LOCATED: Vatican City
NOT LOCATED: Tokyo
NOT LOCATED: Tagsatzung
MORE THAN ONE LOCATION: DESCONOCIDO
MORE THAN ONE LOCATION: Lima
NOT LOCATED: Monaco
NOT LOCATED: Persépolis
MORE THAN ONE LOCATION: Brunswick
MORE THAN ONE LOCATION: Kingston
NOT LOCATED: Brasília
CPU times: user 5.85 s, sys: 169 ms, total: 6.02 s
Wall time: 1min 44s


In [11]:
locs['Kingdom of England'] = (51.5073509, -0.1277583)
locs['Monaco'] = (43.7308084, 7.4225881)
locs["St. John's"] = (47.5615096, -52.7125768)
locs['Batavia, Dutch East Indies'] = (-8.792258, 99.0179762)
locs['Hong Kong'] = (22.3595965, 114.0004413)
locs['Tagsatzung'] = (47.3774336, 8.466504)
locs['Gibraltar'] = (36.1319755, -5.3704699)
locs['Singapore'] = (1.3147268, 103.7065911)
locs['Jakarta'] = (-8.792258, 99.0179762)
locs['Brasília'] = (-15.7217509, -48.0084478)
locs['Vatican City'] = (41.9038163, 12.4476731)
locs['Tokyo'] = (35.6732615, 139.5699618)
locs['Brussels'] = (50.854954, 4.3051791)
locs['Persépolis'] = (31.1625176, -49.7526341)

In [12]:
%%time

def process_geolocation_capital_autor(x):
    c = x['CAPITAL_AUTOR']
    return locs.get(c, (0, 0))

autores_df['GEOLOC_CAPITAL_AUTOR'] = autores_df.apply(process_geolocation_capital_autor, axis=1)

CPU times: user 2.15 s, sys: 9.93 ms, total: 2.16 s
Wall time: 2.16 s


In [13]:
%%time

def process_geolocation_capital_ejemplar(x):
    c = x['CAPITAL_EJEMPLAR']
    return locs.get(c, (0, 0))

autores_df['GEOLOC_CAPITAL_EJEMPLAR'] = autores_df.apply(process_geolocation_capital_ejemplar, axis=1)

CPU times: user 1.57 s, sys: 15.4 ms, total: 1.59 s
Wall time: 1.58 s


# Saving results

In [14]:
autores_df.to_csv('autores7.csv', index=False)